In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import linear_shrinkage
import Util
from Util import evaluate_curret, get_invest_period
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os
import time
import sklearn.covariance
import pickle
from collections import namedtuple, defaultdict
from subprocess import Popen
from multiprocessing import Pool

hi


In [2]:
util_loaded_data = Util.load_data()
ret, ret_nonan, univ, tradeidx, dates = util_loaded_data

## For saving to pastRets (pastRet and KTcov) and outRets

In [4]:
assert os.path.exists('./pastRets')
assert os.path.exists('./outRets')

In [5]:
def save_past_information(T, N, util_loaded_data, save_KT = True):
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    pastRets = []
    KT_covs = []
    
    KT_time = 0
    for h in range(len(univ)):
        print(h, end = ' ')
        
        pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
        pastRetFileName = "pastRets/{}_{}_{}_pastRet.pkl".format(T, N, h)
        with open(pastRetFileName, 'wb') as f:
            pickle.dump(pastRet, f)
        pastRets.append(pastRet)
        
        if save_KT:
            #start = time.time()
            KT_cov = Util.kendall_cov(pastRet)
            KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
            with open(KT_cov_file_name, 'wb') as f:
                pickle.dump(KT_cov, f)
            KT_covs.append(KT_cov)
            end = time.time()
            #print(end-start)
    
    with open("pastRets/{}_{}_pastRets.pkl".format(T,N), 'wb') as f:
        pickle.dump(pastRets, f)

    if save_KT:
        with open("pastRets/{}_{}_KTcovs.pkl".format(T,N), 'wb') as f:
            pickle.dump(KT_covs, f)

In [6]:
#for old purposes
# for T in [25, 50, 100, 200, 500, 1260]:
#     for N in [25, 50, 100, 200, 500, 1000]:
#         save_past_information(T, N, util_loaded_data, save_KT = False)

In [7]:
#for one-off
save_past_information(1260,200, util_loaded_data, save_KT=True)

0 1 

KeyboardInterrupt: 

In [9]:
#for saving KT covs in parallel
def save_KT_cov(h):
    KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
    if os.path.exists(KT_cov_file_name):
        return
    print(h)
    if h == 0:
        print("N={}, T={}".format(N,T))
    start = time.time()
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
    KT_cov = Util.kendall_cov(pastRet)
    KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
    with open(KT_cov_file_name, 'wb') as f:
        pickle.dump(KT_cov, f)
    end = time.time()
    print(end-start)

def consolidate_KT_cov(N,T):
    KT_covs = []
    for h in range(360):
        fname = "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
        with open(fname, 'rb') as f:
            cov = pickle.load(f)
            KT_covs.append(cov)
    with open("pastRets/{}_{}_KTcovs.pkl".format(T,N), 'wb') as f:
            pickle.dump(KT_covs, f)

for N, T in [(500,500)]:
    pool = Pool(8)
    pool.map(save_KT_cov,range(360))
    consolidate_KT_cov(N,T)

302
290
314
326
350
338
66.83202981948853
351
67.26308679580688
315
67.41494822502136
291
67.55869388580322
339
67.59805917739868
303
67.60312795639038
327
72.60824799537659
352
72.53743004798889
292
72.85452198982239
316
72.74070715904236
304
72.92232394218445
340
73.15842890739441
328
70.96964406967163
353
70.84678387641907
293
71.04621696472168
317
71.05210614204407
305
71.33685088157654
341
71.40234327316284
329
71.16410970687866
354
70.74974179267883
318
71.3763039112091
294
71.38018584251404
306
71.06614899635315
342
71.37782192230225
330
71.7751202583313
355
71.50338315963745
319
71.5138931274414
295
71.44897413253784
343
71.61617803573608
307
72.0990378856659
331
71.09482502937317
320
71.43590688705444
356
71.00619006156921
308
71.73735523223877
296
71.16443204879761
344
71.5461859703064
332
71.7401978969574
321
70.85062289237976
297
72.02067732810974
357
71.4175009727478
345
72.03820490837097
309
72.1158549785614
333
70.7034420967102
322
70.79375386238098
298
71.0884096622467


## After this point is the algorithms and experiments, above was just generating data

In [9]:
from estimators import (MTP2_wrapper, MTP2_cov_wrapper,
                        LRPS_wrapper,
                        CLIME_wrapper, CLIME_cov_wrapper,
                        old_LS_wrapper, LS_wrapper, NLS_wrapper,
                        POET_wrapper, POET_5_wrapper,
                        glasso_wrapper,
                        get_AFM_estimator)

In [15]:
all_methods = { #all time estimates for N = 100
    'MTP2_KT': MTP2_cov_wrapper,
    # 'CLIME_KT': CLIME_cov_wrapper
    #'glasso': glasso_wrapper, #2 seconds
    # 'MTP2': MTP2_wrapper, #25 seconds
    # 'CLIME': CLIME_wrapper,
    #'LS': LS_wrapper, #0.5 seconds
    #'old_LS_cov': old_LS_cov_wrapper,
    #'old_LS': old_LS_wrapper,
    #'NLS':  NLS_wrapper, #5 seconds
    #'LRPS': LRPS_wrapper, #7 seconds
    #'AFM_NL': get_AFM_estimator(5, 'NLS', tradeidx), #4 seconds
    #'AFM_LS': get_AFM_estimator(5, 'LS', tradeidx), #0.5 seconds
    #'POET': POET_wrapper,
    #'POET_5': POET_5_wrapper,
    #'AFM_POET': None,
    #'equiweight': None,
    #'POET': POET_wrapper
}

In [ ]:
# For CLIME KT
100 is 8 seconds
200 is 88 seconds
500 is 88 * (500/200)**2 * (88/(8*4)) = 1512 seconds = 25 minutes...
25 * 360 / 7 = 

In [ ]:
# MTP KT is for N=500 is:
1 min (55 seconds) => 6 hrs

### Time estimate

In [16]:
T = 1260
N = 500
with open('pastRets/{}_{}_KTcovs.pkl'.format(T,N), 'rb') as f:
    pastRets = pickle.load(f)
    
timing_dict = {}
for method_name, m_func in all_methods.items():
    h = 0
    print(method_name, end = ' ')
    args = []
    if method_name == "CLIME_KT":
        args = [T]
    start = time.time()
    cov = m_func(pastRets[h], *args)
    end = time.time()
    print(end-start)
    timing_dict[method_name] = end-start

MTP2_KT 56.22209286689758


In [ ]:
print(timing_dict.items())
print("Total hours: ", sum(timing_dict.values()))

### Run versions

In [17]:
assert os.path.exists("./pickle")

In [ ]:
N_T_list = [(500,1260)]

for N, T in N_T_list:
    print(N,T)
    if os.path.isfile('pastRets/{}_{}_KTcovs.pkl'.format(T,N)):
        with open('pastRets/{}_{}_KTcovs.pkl'.format(T,N), 'rb') as f:
            pastCovs = pickle.load(f)
    else:
        assert False
    
run_name = "first_server_run"
KT_or_not = False
for N, T in N_T_list:
    print(N,T)
    with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
        pastRets = pickle.load(f)
        assert len(pastRets) == 360

    def get_covs_all_methods(h):
        print("WORKING ON h={}".format(h))
        for method_name, method in all_methods.items():
            args = []
            if method_name == 'CLIME_KT':
                args = [T]
            cov = method(cov=pastCovs[h], *args)
            fname = 'pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name)
            with open(fname, 'wb') as f:
                pickle.dump(cov, f)
    
    pool = Pool(7)
    pool.map(get_covs_all_methods, range(360))

500 1260
500 1260
WORKING ON h=13
WORKING ON h=26
WORKING ON h=39
WORKING ON h=0
WORKING ON h=52
WORKING ON h=65
WORKING ON h=78
WORKING ON h=14
WORKING ON h=1
WORKING ON h=27
WORKING ON h=79
WORKING ON h=40
WORKING ON h=53
WORKING ON h=66


In [ ]:
def get_portfolio_to_period(end_h, all_methods, T, N, KT_or_not, run_name):
    with open('outRets/{}_outRets.pkl'.format(N), 'rb') as f:
        outRets = pickle.load(f)
    for h in range(end_h):
        print(h, end= ' ')
        m = get_momentum_signal(h, N, univ, tradeidx, ret)
        b = np.mean(m)
        for method_name in iter_method_names:
            if h == 0:
                print('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name))
            with open('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name), 'rb') as f:
                cov = pickle.load(f)
            if cov is None or cov == 'FAIL':
                curret = None
            else:
                w = optimal_weights_momentum(m, cov, b)
                curret = retConstShare(outRets[h], w)
            method_to_portfolio[method_name].append(curret)
    return method_to_portfolio

In [ ]:
method_to_portfolio = get_portfolio_to_period(360, all_methods, 1260, 500, False, "first_server_run")
pname = 'portfolios_IR/{}_{}_{}_{}_{}'.format(T,N,method_name,KT_or_not,run_name)
with open(pname.format(1260,500,"MTP2_KT", False, "first_server_run"), "wb") as f:
    pickle.dump(method_to_portfolio['MTP2_KT'], f)
with open(pname.format(1260,500,"CLIME_KT", False, "first_server_run"), "wb") as f:
    pickle.dump(method_to_portfolio['CLIME_KT'], f)